In [29]:
import pandas as pd
import numpy as np
import random
import math

In [90]:
drop_list = ['does_not_want_to_play_with', 'competitiveness_level', 'fitness_level']

players_df = pd.read_csv('./players.csv').drop(drop_list, axis=1)

players_df

,name,skill,pairing_preference,gender,captain
0,a,0.57,b,m,yes
1,b,0.69,a,f,no
2,c,0.38,d,n,no
3,d,0.86,c,m,no
4,e,0.27,f,f,no
5,f,1.00,e,n,no
6,g,0.13,h,m,no
7,h,0.09,g,f,no
8,I,0.19,j,n,no
9,j,0.70,I,m,no


In [101]:
def make_balanced_teams(players_df, num_teams, iter_num):
    """Make balanced sport teams using an optimization-based approach.

    Args:
        players_df (pandas.DataFrame): A DataFrame with columns "name", "gender", "skill", and "captain" representing the players.
        num_teams (int): The number of teams to create.

    Returns:
        teams (list): A list of team lists, where each team list contains dictionaries representing players.
        metrics (list): A list of dictionaries, where each dictionary contains the gender and skill balance metrics for a team.
    """
    # Split players into male, female, and non-binary DataFrames
    male_players = players_df[players_df['gender'] == 'm']
    female_players = players_df[players_df['gender'] == 'f']
    nonbinary_players = players_df[players_df['gender'] == 'n']
    
    # Create a list of dictionaries representing the players
    players = []
    for gender_df in [male_players, female_players, nonbinary_players]:
        for _, row in gender_df.iterrows():
            player = {
                'name': row['name'],
                'gender': row['gender'],
                'skill': row['skill'],
                'captain': row['captain']
            }
            players.append(player)
    
    # Randomly assign players to teams, with one captain per team
    captains = [player for player in players if player['captain'] == 'yes']
    non_captains = [player for player in players if player['captain'] == 'no']
    
    # Calculate the number of players per team
    num_players_per_team = math.ceil(len(non_captains) / num_teams)
    
    # Shuffle the list of non-captains
    random.shuffle(non_captains)
    
    # Split the list of non-captains into groups of num_players_per_team
    non_captains_groups = [non_captains[i:i+num_players_per_team] for i in range(0, len(non_captains), num_players_per_team)]
    
    # Assign each group of non-captains to a team
    teams = []
    for i in range(num_teams):
        team = []
        teams.append(team)
        team.append(random.choice(captains))
        team += non_captains_groups[i]
       
    # Balance the teams by swapping players
    for i in range(iter_num):  # Maximum number of iterations
        # Choose two random teams
        team1 = random.choice(teams)
        team2 = random.choice(teams)
    
        # Choose two random non-captain players from the teams
        team1_non_captains = [p for p in team1 if p not in captains]
        team2_non_captains = [p for p in team2 if p not in captains]
        if not team1_non_captains or not team2_non_captains:
            continue
    
    player1 = random.choice(team1_non_captains)
    player2 = random.choice(team2_non_captains)
    
    # Calculate the skill and gender balance of the teams if the players are swapped
    team1_skill = sum([p['skill'] for p in team1 if p is not player1]) + player2['skill']
    team2_skill = sum([p['skill'] for p in team2 if p is not player2]) + player1['skill']
    team1_gender = pd.DataFrame(team1 + [player2])['gender'].value_counts()
    team2_gender = pd.DataFrame(team2 + [player1])['gender'].value_counts()
                
    # Swap the players if the teams are more balanced
    if abs(team1_skill - team2_skill) <= 1 and max(team1_gender.max(), team2_gender.max()) <= len(team1) // 2:
        if player1 in team1:
            team1.remove(player1)
            team1.append(player2)
        if player2 in team2:
            team2.remove(player2)
            team2.append(player1)
        # Recalculate the gender and skill balance metrics of the teams after swapping players
        team1_skill = sum([p['skill'] for p in team1])
        team2_skill = sum([p['skill'] for p in team2])
        team1_gender = pd.DataFrame(team1)['gender'].value_counts()
        team2_gender = pd.DataFrame(team2)['gender'].value_counts()

    # Calculate gender and skill balance metrics of the teams
    metrics = []
    for team in teams:
        team_gender = pd.DataFrame(team)['gender'].value_counts()
        team_skill = sum([p['skill'] for p in team])
        metric = {
            'male_ratio': round(team_gender['m'] / len(team), 2),
            'nonbinary_ratio': round(team_gender['n'] / len(team), 2),
            'female_ratio': round(team_gender['f'] / len(team), 2),
            'avg_skill': round(team_skill / len(team), 2)
        }
        metrics.append(metric)

    # Print gender and skill balance metrics
    for i, metric in enumerate(metrics):
        print(f'Team {i+1}: Male Ratio: {metric["male_ratio"]}, Nonbinary Ratio: {metric["nonbinary_ratio"]}, Female Ratio: {metric["female_ratio"]}, Average Skill: {metric["avg_skill"]}')

    return teams

In [103]:
make_balanced_teams(players_df, 4, 10)

Team 1: Male Ratio: 0.3, Nonbinary Ratio: 0.5, Female Ratio: 0.2, Average Skill: 0.73
Team 2: Male Ratio: 0.2, Nonbinary Ratio: 0.4, Female Ratio: 0.4, Average Skill: 0.46
Team 3: Male Ratio: 0.3, Nonbinary Ratio: 0.2, Female Ratio: 0.5, Average Skill: 0.43
Team 4: Male Ratio: 0.5, Nonbinary Ratio: 0.1, Female Ratio: 0.4, Average Skill: 0.66


[[{'name': 'gg', 'gender': 'm', 'skill': 0.7, 'captain': 'yes'},
  {'name': 'hh', 'gender': 'f', 'skill': 0.8, 'captain': 'no'},
  {'name': 'f', 'gender': 'n', 'skill': 1.0, 'captain': 'no'},
  {'name': 'c', 'gender': 'n', 'skill': 0.38, 'captain': 'no'},
  {'name': 'j', 'gender': 'm', 'skill': 0.7, 'captain': 'no'},
  {'name': 'm', 'gender': 'm', 'skill': 0.66, 'captain': 'no'},
  {'name': 'ii', 'gender': 'n', 'skill': 0.9, 'captain': 'no'},
  {'name': 'kk', 'gender': 'n', 'skill': 0.85, 'captain': 'no'},
  {'name': 'e', 'gender': 'f', 'skill': 0.27, 'captain': 'no'},
  {'name': 'jj', 'gender': 'n', 'skill': 1.0, 'captain': 'no'}],
 [{'name': 'aa', 'gender': 'f', 'skill': 0.1, 'captain': 'yes'},
  {'name': 'r', 'gender': 'n', 'skill': 0.82, 'captain': 'no'},
  {'name': 'mm', 'gender': 'f', 'skill': 0.45, 'captain': 'no'},
  {'name': 'cc', 'gender': 'n', 'skill': 0.3, 'captain': 'no'},
  {'name': 'y', 'gender': 'm', 'skill': 0.75, 'captain': 'no'},
  {'name': 'u', 'gender': 'n', 'skill

In [ ]:
# Current issues
# sometimes teams have no m, f, or n so we get key errors for metrics for example
# no player preferences yet